# Table scraping using pandas and storing into database

In [1]:
import pandas as pd
import ssl
from sqlalchemy import create_engine, inspect

ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
# scrape tables using pandas
url = 'https://en.wikipedia.org/wiki/List_of_NCAA_Division_I_FBS_football_stadiums'
stadiums_df = pd.read_html(url)[0]
stadiums_df

Image                                            Stadium         City  \
0      NaN                             Aggie Memorial Stadium   Las Cruces   
1      NaN                                          Alamodome  San Antonio   
2      NaN             Alaska Airlines Field at Husky Stadium      Seattle   
3      NaN                                 Albertsons Stadium        Boise   
4      NaN                                  Allegiant Stadium     Paradise   
..     ...                                                ...          ...   
125    NaN  Veterans Memorial Stadium at Larry Blakeney Field         Troy   
126    NaN                                      Waldo Stadium    Kalamazoo   
127    NaN                      Warren McGuirk Alumni Stadium       Hadley   
128    NaN    Wayne Day Family Field at Carter–Finley Stadium      Raleigh   
129    NaN                             Williams-Brice Stadium     Columbia   

    State              Team     Conference                           Capacity  \
0      NM  New Mexico State    Independent                          30,343[1]   
1      TX              UTSA          C-USA                              65000   
2      WA        Washington         Pac-12                          70,083[2]   
3      ID       Boise State  Mountain West                          36,387[3]   
4      NV              UNLV  Mountain West                              65000   
..    ...               ...            ...                                ...   
125    AL              Troy       Sun Belt                              30402   
126    MI  Western Michigan            MAC                        30,200[152]   
127    MA             UMass    Independent  17,000(expandable to 21,430[154])   
128    NC          NC State            ACC                        57,583[155]   
129    SC    South Carolina            SEC                        77,559[156]   

                                              Record1      Built   Expanded 2  \
0              32,993(September 26, 1998 vs. UTEP)[1]    1978[1]      2005[1]   
1    56,743(September 3, 2011 vs. Northeastern State)       1993          NaN   
2                 76,125(September 23, 1995 vs. Army)       1920         2013   
3               36,864(September 20, 2012 vs. BYU)[4]    1970[3]      2012[3]   
4                                                 NaN       2020          NaN   
..                                                ...        ...          ...   
125  29,013(September 15, 2012 vs. Mississippi State)       1950  2003 & 2018   
126      36,361(September 16, 2000 vs. Indiana State)  1939[153]    2003[153]   
127      20,000(November 25, 1972 vs. Boston College)       1965         2014   
128                            57,583(multiple times)       1966         2006   
129          85,199(October 6, 2012 vs. Georgia)[156]  1934[156]    1996[156]   

                      Surface  
0         S5-M Synthetic Turf  
1                  SportField  
2                   FieldTurf  
3                  Smurf Turf  
4                   FieldTurf  
..                        ...  
125                  ProGrass  
126                 FieldTurf  
127                 FieldTurf  
128  Tifway 419 Bermuda Grass  
129                     Grass  

[130 rows x 11 columns]

In [3]:
# scrape tables using pandas
url = 'https://en.wikipedia.org/wiki/NCAA_Division_I_FBS_football_win-loss_records'
win_loss_record= pd.read_html(url)[2]
win_loss_record

Team  Won  Lost  Tied   Pct.  Years  Total Games     Conference
0     Ohio State 1  931   327    53  0.730    130         1311        Big Ten
1        Alabama 2  929   331    43  0.729    125         1303            SEC
2     Notre Dame 3  918   328    42  0.729    130         1288    Independent
3      Boise State  465   172     2  0.729     52          639  Mountain West
4         Michigan  964   350    36  0.727    140         1350        Big Ten
..             ...  ...   ...   ...    ...    ...          ...            ...
125        UNLV 11  236   364     4  0.394     52          604  Mountain West
126     Kent State  353   570    28  0.386     97          951            MAC
127            FIU   79   143     0  0.356     18          222          C-USA
128      Charlotte   34    62     0  0.354      9           96          C-USA
129  Georgia State   42    87     0  0.326     10          129       Sun Belt

[130 rows x 8 columns]

In [4]:
#create sqlite database and connect to it
database_path = "Resources/college_football_db.sqlite"
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

In [5]:
# store stadiums dataframe into database
stadiums_df.to_sql('stadiums', conn, if_exists='replace')

In [119]:
# store dataframe into database
win_loss_record.to_sql('win_loss_record', conn, if_exists='replace')

In [120]:
# close connection
conn.close()

# Read tables from database into dataframe

In [109]:
# connect to sqlite database
database_path = "Resources/college_football_db.sqlite"
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

In [112]:
# view tables in database
inspector = inspect(engine)
inspector.get_table_names()

['stadiums', 'win_loss_record']

In [101]:
# once connected read in table from database (stadiums table)
stadium_df = pd.read_sql_table('stadiums', conn)
stadium_df.head()

index  Image                                 Stadium         City State  \
0      0    NaN                  Aggie Memorial Stadium   Las Cruces    NM   
1      1    NaN                               Alamodome  San Antonio    TX   
2      2    NaN  Alaska Airlines Field at Husky Stadium      Seattle    WA   
3      3    NaN                      Albertsons Stadium        Boise    ID   
4      4    NaN                       Allegiant Stadium     Paradise    NV   

               Team     Conference   Capacity  \
0  New Mexico State    Independent  30,343[1]   
1              UTSA          C-USA      65000   
2        Washington         Pac-12  70,083[2]   
3       Boise State  Mountain West  36,387[3]   
4              UNLV  Mountain West      65000   

                                            Record1    Built Expanded 2  \
0            32,993(September 26, 1998 vs. UTEP)[1]  1978[1]    2005[1]   
1  56,743(September 3, 2011 vs. Northeastern State)     1993       None   
2               76,125(September 23, 1995 vs. Army)     1920       2013   
3             36,864(September 20, 2012 vs. BYU)[4]  1970[3]    2012[3]   
4                                              None     2020       None   

               Surface  
0  S5-M Synthetic Turf  
1           SportField  
2            FieldTurf  
3           Smurf Turf  
4            FieldTurf

In [111]:
#(win_loss_record table)
stadium_df = pd.read_sql_table('win_loss_record', conn)
conn.close()
stadium_df.head()

index          Team  Won  Lost  Tied   Pct.  Years  Total Games  \
0      0  Ohio State 1  931   327    53  0.730    130         1311   
1      1     Alabama 2  929   331    43  0.729    125         1303   
2      2  Notre Dame 3  918   328    42  0.729    130         1288   
3      3   Boise State  465   172     2  0.729     52          639   
4      4      Michigan  964   350    36  0.727    140         1350   

      Conference  
0        Big Ten  
1            SEC  
2    Independent  
3  Mountain West  
4        Big Ten